# ME 592 Final Project
## Jake Bergfeld, Mohammad Rashid Mohammad Shoaib
#### Engineering Image Analysis - Automated Chest X-ray classifier

##### Gathering data from Kaggle - Link to data: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

In [1]:
# !pip install kaggle
# !mkdir .kaggle  #naming required by kaggle API, creates a hidden folder
# !cp /home/exouser/Downloads/kaggle.json /home/exouser/.kaggle/kaggle.json

##### <u>Confirming location of Kaggle API token was moved successfully

In [2]:
# !cd .kaggle && ls

##### <u>Downloading the specific dataset and confirming locations

In [1]:
# !kaggle datasets list -s 'Chest X-ray images'

In [4]:
# !kaggle datasets download -d 'paultimothymooney/chest-xray-pneumonia'

In [5]:
# !sudo apt-get install unzip
# !unzip chest-xray-pneumonia.zip /home/exouser/ME592/Final Project

### <u>Data Information: 

### Input image size varies
    
##### The 2 classes to predict are:
 - Normal/Healthy lungs
 - Pneumonia

##### *Formula to calculate the number of parameters in a CNN:*
   -  Convolutional layer: (in_channels x out_channels x kernel_height x kernel_width) + out_channels
   -  Batch normalization layer: 2 x num_features
   -  ReLU activation layer: 0 (no parameters)
   -  Max pooling layer: 0 (no parameters)
   -  Fully connected layer: (in_features x out_features) + out_features

### <u>Library Imports & Transforming Data

In [2]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import shutil
import random

In [3]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.empty_cache()

cuda


In [4]:
#Transform the input images
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
# #Only run this once to generate test and train data and create separate folders for each

# import os
# import random
# import shutil

# # Set the path to the 'Train & Test' folder
# folder_path = '/home/exouser/data/imgs/Train & Test'

# # Set the path to the 'Train' folder
# train_path = '/home/exouser/data/imgs/Train & Test/Train'

# # Set the path to the 'Test' folder
# test_path = '/home/exouser/data/imgs/Train & Test/Test'

# # Loop through each subfolder in the 'Train & Test' folder
# for subfolder in os.listdir(folder_path):

#     # Create a new subfolder in the 'Train' folder with the same name
#     train_subfolder = os.path.join(train_path, subfolder)
#     os.makedirs(train_subfolder, exist_ok=True)

#     # Create a new subfolder in the 'Test' folder with the same name
#     test_subfolder = os.path.join(test_path, subfolder)
#     os.makedirs(test_subfolder, exist_ok=True)

#     # List all the image files in the subfolder
#     images = os.listdir(os.path.join(folder_path, subfolder))
#     random.shuffle(images)

#     # Calculate the number of images to move to the 'Train' folder
#     split_index = int(len(images) * 0.7)

#     # Move the first 70% of images to the 'Train' folder
#     for image in images[:split_index]:
#         source = os.path.join(folder_path, subfolder, image)
#         destination = os.path.join(train_subfolder, image)
#         shutil.copyfile(source, destination)

#     # Move the remaining 30% of images to the 'Test' folder
#     for image in images[split_index:]:
#         source = os.path.join(folder_path, subfolder, image)
#         destination = os.path.join(test_subfolder, image)
#         shutil.copyfile(source, destination)


### <u>Dataloader and Pre-training

In [61]:
#Create a dataloader

#Directory path for the training & test images
train_path='/home/exouser/ME592/Final Project/chest-xray-pneumonia/train'
test_path='/home/exouser/ME592/Final Project/chest-xray-pneumonia/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=16, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=16, shuffle=True
) 


In [62]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)


['NORMAL', 'PNEUMONIA']


### <u>Step 1: Build and train a CNN model

In [63]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=1024)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=1024, out_features=num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool(output)

        output = output.view(-1, 128 * 18 * 18)
        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)
        output = self.sigmoid(output)
        
        #Final layer can be a sigmoid activation layer since only using 2 classes. This will help push the network, on/off functionality. Should help with test acc

        return output
        

In [64]:
model=ConvNet(num_classes=2).to(device)


In [73]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
# loss_function=nn.CrossEntropyLoss()
loss_function=nn.BCEWithLogitsLoss()
#Since binary problem, can use Binary cross entropy with logits function instead to perform the sigmoid entropy automatically (numerical stability)


In [74]:
num_epochs=10


In [75]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpeg'))
test_count=len(glob.glob(test_path+'/**/*.jpeg'))


In [76]:
print(train_count)
print(test_count)


5216
624


In [78]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        labels = labels.unsqueeze(1) #ADDING TO ASSIST BCELoss & Sigmoid layer
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs.float(),labels.float())
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    # print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))

    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.float().unsqueeze(1).cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

ValueError: Target size (torch.Size([16, 1])) must be the same as input size (torch.Size([16, 2]))

### <u>Now using unlabeled images to evaluate the network:

In [82]:
#Loading more libraries
import torch
import torch.nn as nn
import numpy as np
import torch.functional as F
import os
import pathlib
import glob
import cv2
from torch.autograd import Variable
from torchvision.transforms import transforms
from torchvision.models import squeezenet1_1
from io import open
from PIL import Image

In [83]:
pred_path='/home/exouser/ME592/Final Project/chest-xray-pneumonia/val'

In [50]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=2)
model.load_state_dict(checkpoint)
model.eval()

RuntimeError: Error(s) in loading state_dict for ConvNet:
	size mismatch for fc2.weight: copying a param with shape torch.Size([2, 1024]) from checkpoint, the shape in current model is torch.Size([1, 1024]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([2]) from checkpoint, the shape in current model is torch.Size([1]).

In [ ]:
#Transform the prediction input images
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#Prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    image_tensor=transformer(image).float()
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor=image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    
    return pred

In [ ]:
images_path=glob.glob(pred_path+'/*.jpg')

In [ ]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [ ]:
pred_dict

In [38]:
class ConvNet10M(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet10M, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.relu1 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=128)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=256 * 18 * 18, out_features=4096)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=4096, out_features=2048)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(in_features=2048, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool(output)

        output = output.view(-1, 256 * 18 * 18)
        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)
        output = self.relu5(output)
        output = self.fc3(output)

        return output


In [39]:
model=ConvNet10M(num_classes=2).to(device)

In [ ]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))

In [ ]:
print(train_count)

In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    # print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))

    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       